# Part 5.2: TARNet 与 DragonNet - 深度因果推断的演进

## 学习目标

1. 理解 TARNet 的架构设计和 Factual Loss
2. 掌握 DragonNet 的三头架构和倾向得分正则化
3. 理解 Targeted Regularization 的原理
4. 通过对比实验理解两者的区别
5. 学会在实践中选择合适的模型

---

## 共同背景：深度学习用于因果推断

### 为什么需要深度学习？

传统因果推断方法（如 IPW, Matching）面临的挑战：
- **高维特征**: 现代应用中特征维度可能很高（图像、文本、用户行为）
- **非线性关系**: 特征与结果之间的关系可能高度非线性
- **表示学习**: 需要自动学习有效的特征表示

### 核心问题：反事实无法观测

对于每个样本 $i$:
- 如果 $T_i=1$，我们只观测到 $Y_i(1)$，$Y_i(0)$ 是反事实
- 如果 $T_i=0$，我们只观测到 $Y_i(0)$，$Y_i(1)$ 是反事实

这就是**因果推断的根本挑战**！

In [ ]:
# 导入必要的库
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from typing import Tuple, Dict
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

# 绘图设置
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12
# 使用兼容的样式
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except:
    plt.style.use('seaborn-whitegrid' if 'seaborn-whitegrid' in plt.style.available else 'default')

print(f"PyTorch 版本: {torch.__version__}")
print("环境准备完成！")

---

# Part 1: TARNet - 开山之作

## TARNet: Treatment-Agnostic Representation Network

TARNet 是 2017 年提出的开创性工作，首次将深度学习系统地应用于因果效应估计。

### 核心设计

1. **共享表示层**：学习对处理组和控制组都有用的特征表示
2. **双头输出**：分别预测 Y(0) 和 Y(1)

$$X \xrightarrow{\Phi} \text{Representation} \xrightarrow{\begin{cases} h_0 \to \hat{Y}(0) \\ h_1 \to \hat{Y}(1) \end{cases}}$$

### 生活化类比：双语翻译官

想象你是一个翻译官，需要翻译中文文章给两类读者：
- **英语读者** (处理组)
- **法语读者** (控制组)

**传统方法 (T-Learner)**：
- 雇佣两个独立的翻译官
- 每个翻译官只懂一种语言
- 效率低，且无法利用共同知识

**TARNet 方法**：
- 一个翻译官先理解文章核心含义 (共享表示)
- 然后分别翻译成英语和法语 (双头输出)
- 效率高，且能利用语言间的共同结构

### TARNet 架构图

```
        ┌─────────────────────────────────────────┐
        │         共享表示层 (Shared Repr)          │
        │  X → [Hidden] → [Hidden] → Φ(X)         │
        └─────────────────┬───────────────────────┘
                          │
            ┌─────────────┴─────────────┐
            ▼                           ▼
    ┌───────────────┐           ┌───────────────┐
    │    Head 0     │           │    Head 1     │
    │ Φ(X) → Ŷ(0)  │           │ Φ(X) → Ŷ(1)  │
    └───────────────┘           └───────────────┘
            │                           │
            ▼                           ▼
      控制组预测                    处理组预测
```

## Factual Loss: 关键训练策略

### 核心挑战：反事实无法观测

对于每个样本，我们只能观测到一个结果：
- 如果 $T=1$，我们只观测到 $Y(1)$
- 如果 $T=0$，我们只观测到 $Y(0)$

所以我们只能在 **观测到的结果** 上计算损失！

### Factual Loss 公式

$$L_{\text{factual}} = \frac{1}{n}\sum_{i=1}^{n} (Y_i - \hat{Y}_i^{\text{factual}})^2$$

其中：
$$\hat{Y}_i^{\text{factual}} = \begin{cases} \hat{Y}_i(1) & \text{if } T_i = 1 \\ \hat{Y}_i(0) & \text{if } T_i = 0 \end{cases}$$

In [ ]:
# 练习 1.1: 理解 TARNet 架构

class SimpleTARNet(nn.Module):
    """
    简化版 TARNet
    
    架构:
    X -> [Shared Representation] -> Phi(X)
                                      |
                    +----------------+----------------+
                    |                                 |
                [Head 0]                         [Head 1]
                    |                                 |
                  Y(0)                              Y(1)
    """
    
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int = 50,
        repr_dim: int = 25
    ):
        super().__init__()
        
        # 共享表示层
        self.representation = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, repr_dim),
            nn.ReLU(),
        )
        
        # 控制组输出头 (Y0)
        self.head0 = nn.Sequential(
            nn.Linear(repr_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1),
        )
        
        # 处理组输出头 (Y1)
        self.head1 = nn.Sequential(
            nn.Linear(repr_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1),
        )
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        前向传播
        
        Returns:
            (y0_pred, y1_pred, representation)
        """
        phi = self.representation(x)
        y0 = self.head0(phi)
        y1 = self.head1(phi)
        
        return y0, y1, phi
    
    def predict_ite(self, x: torch.Tensor) -> torch.Tensor:
        """预测个体处理效应 ITE = Y(1) - Y(0)"""
        y0, y1, _ = self.forward(x)
        return y1 - y0

# 测试架构
model = SimpleTARNet(input_dim=5)
X_sample = torch.randn(10, 5)
y0, y1, phi = model(X_sample)
print(f"TARNet 输出形状: Y0={y0.shape}, Y1={y1.shape}, Phi={phi.shape}")

In [ ]:
# 练习 1.2: Factual Loss

def compute_factual_loss(
    y_true: torch.Tensor,
    t_true: torch.Tensor,
    y0_pred: torch.Tensor,
    y1_pred: torch.Tensor
) -> torch.Tensor:
    """
    计算 Factual Loss
    
    关键思想: 只在观测到的结果上计算损失
    - 如果 T=1, 损失 = (Y - Y1_pred)^2
    - 如果 T=0, 损失 = (Y - Y0_pred)^2
    """
    # 根据处理状态选择预测值
    t_expanded = t_true.unsqueeze(1) if len(t_true.shape) == 1 else t_true
    y_pred = torch.where(t_expanded == 1, y1_pred, y0_pred)
    
    # 计算 MSE
    y_true_expanded = y_true.unsqueeze(1) if len(y_true.shape) == 1 else y_true
    loss = torch.mean((y_true_expanded - y_pred)**2)
    
    return loss

# 测试
y_true = torch.FloatTensor([1.0, 2.0, 3.0])
t_true = torch.FloatTensor([1.0, 0.0, 1.0])
y0_pred = torch.FloatTensor([[1.5], [2.0], [2.5]])
y1_pred = torch.FloatTensor([[1.0], [2.5], [3.0]])

loss = compute_factual_loss(y_true, t_true, y0_pred, y1_pred)
print(f"Factual Loss: {loss.item():.4f}")

---

# Part 2: DragonNet - 三头龙的进化

## 从 TARNet 到 DragonNet: 一个巧妙的升级

如果翻译官同时还能**判断客户来自哪个国家**（倾向得分），翻译效果会不会更好呢？

这就是 **DragonNet** 的核心思想！

## DragonNet 的直觉: 三头龙的故事

### 问题回顾

在因果推断中，**混淆**是最大的敌人。想象你在研究「广告是否增加用户购买」：

- 经常购物的用户**更容易看到广告**（算法推荐）
- 经常购物的用户**本来就会买更多**

这就是**选择偏差**：处理分配不是随机的，而是依赖于混淆因子。

### TARNet 的局限

TARNet 只学习「预测结果」，但不显式地理解「谁会被处理」。

### DragonNet 的解决方案

DragonNet 添加了**第三个头**——倾向得分头！

```
                       🧠 共享表示层
                           |                    
       +------------------+-------------------+
       |                  |                   |
    🎯 Y(0)头         🎯 Y(1)头          📊 倾向得分头
       |                  |                   |
  控制组结果预测    处理组结果预测      "谁会被处理?"
```

为什么叫「龙网」（DragonNet）？因为这三个头就像传说中的**三头龙**！

### 关键洞察

添加倾向得分头有什么好处？

1. **正则化作用**: 强迫表示层同时学会「预测结果」和「预测处理」
2. **平衡表示**: 让表示层捕获与**处理分配相关的信息**，这正是混淆因子！
3. **Targeted Regularization**: 用倾向得分来调整损失函数，减少偏差

## DragonNet 核心公式

### 架构

$$\Phi(X) = f_{\text{repr}}(X) \quad \text{(共享表示)}$$

$$\hat{Y}(0) = h_0(\Phi(X)), \quad \hat{Y}(1) = h_1(\Phi(X)) \quad \text{(结果头)}$$

$$\hat{e}(X) = h_e(\Phi(X)) \quad \text{(倾向得分头)}$$

### 复合损失函数

$$\mathcal{L}_{\text{DragonNet}} = \mathcal{L}_{\text{factual}} + \alpha \cdot \mathcal{L}_{\text{propensity}} + \beta \cdot \mathcal{L}_{\text{targeted}}$$

其中：

1. **Factual Loss** (和 TARNet 一样):
$$\mathcal{L}_{\text{factual}} = \frac{1}{N}\sum_{i=1}^{N} \left(Y_i - \left[T_i \cdot \hat{Y}_i(1) + (1-T_i) \cdot \hat{Y}_i(0)\right]\right)^2$$

2. **Propensity Loss** (二分类交叉熵):
$$\mathcal{L}_{\text{propensity}} = -\frac{1}{N}\sum_{i=1}^{N} \left[T_i \log \hat{e}_i + (1-T_i) \log (1-\hat{e}_i)\right]$$

3. **Targeted Regularization** (DragonNet 的创新):
$$\mathcal{L}_{\text{targeted}} = \frac{1}{N}\sum_{i=1}^{N} \left(Y_i - \hat{Y}_i - \epsilon \cdot h_i\right)^2$$

其中：
$$h_i = \frac{T_i}{\hat{e}(X_i)} - \frac{1-T_i}{1-\hat{e}(X_i)}$$

## Targeted Regularization: 魔法公式的直觉

### 那个神秘的 h 是什么？

$$h = \frac{T}{e(X)} - \frac{1-T}{1-e(X)}$$

这个公式来自于**半参数效率理论**和 **TMLE（Targeted Maximum Likelihood Estimation）**。

### 一个直觉解释

想象你在做民意调查，但有些人群更难接触到（比如年轻人不爱接电话）：

- **e(X)** = 某人被调查到的概率（倾向得分）
- **T/e(X)**: 如果被调查到（T=1），用 1/e(X) 加权，相当于「少见的人更重要」
- **-(1-T)/(1-e(X))**: 如果没被调查到（T=0），用反向权重

这个 **h** 本质上是一个**双重鲁棒性**的调整项！

### epsilon 的作用

**epsilon (ε)** 是一个**可学习的标量参数**：

- 它让模型自动学习「需要多少调整」
- 如果模型已经很准，epsilon 会趋近于 0
- 如果有偏差，epsilon 会学到一个非零值来修正

In [ ]:
# 练习 2.1: DragonNet 架构

class SimpleDragonNet(nn.Module):
    """
    简化版 DragonNet - 三头龙网络
    
    与 TARNet 的区别:
    1. 多了一个倾向得分头
    2. 有一个可学习的 epsilon 参数
    3. 使用 ELU 激活函数（原论文推荐）
    """
    
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int = 50,
        repr_dim: int = 25
    ):
        super().__init__()
        
        # 共享表示层 (使用 ELU)
        self.representation = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, repr_dim),
            nn.ELU(),
        )
        
        # 控制组输出头 (Y0)
        self.head0 = nn.Sequential(
            nn.Linear(repr_dim, hidden_dim // 2),
            nn.ELU(),
            nn.Linear(hidden_dim // 2, 1),
        )
        
        # 处理组输出头 (Y1)
        self.head1 = nn.Sequential(
            nn.Linear(repr_dim, hidden_dim // 2),
            nn.ELU(),
            nn.Linear(hidden_dim // 2, 1),
        )
        
        # 倾向得分头 (这是关键！)
        self.propensity_head = nn.Sequential(
            nn.Linear(repr_dim, hidden_dim // 2),
            nn.ELU(),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()  # 输出限制在 [0, 1]
        )
        
        # epsilon 参数 (可学习的标量)
        self.epsilon = nn.Parameter(torch.tensor(0.0))
    
    def forward(self, x: torch.Tensor) -> Tuple:
        """
        前向传播
        
        Returns:
            (y0_pred, y1_pred, propensity, epsilon, representation)
        """
        phi = self.representation(x)
        y0 = self.head0(phi)
        y1 = self.head1(phi)
        propensity = self.propensity_head(phi)
        
        return y0, y1, propensity, self.epsilon, phi
    
    def predict_ite(self, x: torch.Tensor) -> torch.Tensor:
        """预测个体处理效应"""
        y0, y1, _, _, _ = self.forward(x)
        return y1 - y0

# 测试 DragonNet 架构
dragon_model = SimpleDragonNet(input_dim=5)
X_sample = torch.randn(10, 5)
y0, y1, prop, eps, phi = dragon_model(X_sample)

print("DragonNet 架构测试通过! 🐉")
print(f"\n输出形状:")
print(f"  Y0 预测: {y0.shape}")
print(f"  Y1 预测: {y1.shape}")
print(f"  倾向得分: {prop.shape}")
print(f"  表示向量: {phi.shape}")
print(f"\n关键参数:")
print(f"  Epsilon: {eps}")
print(f"  倾向得分范围: [{prop.min():.4f}, {prop.max():.4f}]")

In [ ]:
# 练习 2.2: DragonNet 复合损失函数

def dragonnet_loss(
    y_true: torch.Tensor,
    t_true: torch.Tensor,
    y0_pred: torch.Tensor,
    y1_pred: torch.Tensor,
    propensity: torch.Tensor,
    epsilon: torch.Tensor,
    alpha: float = 1.0,
    beta: float = 1.0
) -> Dict[str, torch.Tensor]:
    """
    DragonNet 复合损失函数
    
    L = L_factual + alpha * L_propensity + beta * L_targeted
    """
    # 确保形状正确
    y0_pred = y0_pred.squeeze()
    y1_pred = y1_pred.squeeze()
    propensity = propensity.squeeze()
    
    # 1. Factual Loss
    y_pred = t_true * y1_pred + (1 - t_true) * y0_pred
    factual_loss = torch.mean((y_true - y_pred) ** 2)
    
    # 2. Propensity Loss (二分类交叉熵)
    eps = 1e-8
    propensity_loss = -torch.mean(
        t_true * torch.log(propensity + eps) + 
        (1 - t_true) * torch.log(1 - propensity + eps)
    )
    
    # 3. Targeted Regularization
    h = t_true / (propensity + eps) - (1 - t_true) / (1 - propensity + eps)
    targeted_reg = torch.mean((y_true - y_pred - epsilon * h) ** 2)
    
    # 总损失
    total_loss = factual_loss + alpha * propensity_loss + beta * targeted_reg
    
    return {
        'total': total_loss,
        'factual': factual_loss,
        'propensity': propensity_loss,
        'targeted': targeted_reg
    }

# 测试损失函数
y_true = torch.FloatTensor([1.0, 2.0, 3.0, 4.0, 5.0])
t_true = torch.FloatTensor([1.0, 0.0, 1.0, 0.0, 1.0])
y0_pred = torch.FloatTensor([1.5, 2.0, 2.5, 4.5, 4.0])
y1_pred = torch.FloatTensor([2.0, 2.5, 3.0, 5.0, 5.5])
propensity = torch.FloatTensor([0.7, 0.3, 0.6, 0.4, 0.8])
epsilon = torch.tensor(0.1)

losses = dragonnet_loss(
    y_true, t_true, y0_pred, y1_pred,
    propensity, epsilon, alpha=1.0, beta=1.0
)

print("损失函数测试通过! 🎉")
print(f"\n各损失项:")
print(f"  Factual Loss: {losses['factual'].item():.4f}")
print(f"  Propensity Loss: {losses['propensity'].item():.4f}")
print(f"  Targeted Reg: {losses['targeted'].item():.4f}")
print(f"  Total Loss: {losses['total'].item():.4f}")

---

# Part 3: 数据生成与实验准备

In [ ]:
# 练习 3.1: 生成强混淆数据

def generate_confounded_data(
    n: int = 2000,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    生成有强混淆的数据
    
    场景: 广告投放效果评估
    - X: 用户特征（购物频率、平台活跃度、消费能力等）
    - T: 是否看到广告（不是随机分配！高价值用户更容易看到）
    - Y: 是否购买
    
    强混淆: 用户特征同时影响「是否看广告」和「是否购买」
    
    DGP (Data Generating Process):
    1. 倾向得分: e(X) = sigmoid(1.0*X₀ + 0.8*X₁ + 0.6*X₂)
    2. 控制组结果: Y(0) = 2.0 + 1.0*X₀ + 0.5*X₁ + ε
    3. 处理效应: τ(X) = 2.0 + 0.5*X₀
    4. 处理组结果: Y(1) = Y(0) + τ(X)
    """
    np.random.seed(seed)
    
    # 用户特征 (5维)
    X = np.random.randn(n, 5)
    # X[:, 0]: 购物频率
    # X[:, 1]: 平台活跃度
    # X[:, 2]: 消费能力
    # X[:, 3]: 浏览历史
    # X[:, 4]: 点击率
    
    # 强混淆的处理分配（高价值用户更容易看到广告）
    propensity = 1 / (1 + np.exp(-(
        1.0 * X[:, 0] +   # 购物频率高 -> 更可能看广告
        0.8 * X[:, 1] +   # 活跃度高 -> 更可能看广告
        0.6 * X[:, 2]     # 消费能力高 -> 更可能看广告
    )))
    
    T = np.random.binomial(1, propensity, n)
    
    # 生成潜在结果（结果也依赖于 X - 这是混淆！）
    noise = np.random.randn(n) * 0.5
    Y0 = 2.0 + 1.0 * X[:, 0] + 0.5 * X[:, 1] + noise  # 修正系数，与文档一致
    
    # 异质性处理效应
    treatment_effect = 2.0 + 0.5 * X[:, 0]  # 基础效应 2, 加上与特征相关的部分
    Y1 = Y0 + treatment_effect
    
    # 观测结果
    Y = np.where(T == 1, Y1, Y0)
    
    return X, T, Y, Y0, Y1

# 生成数据
X, T, Y, Y0, Y1 = generate_confounded_data(n=2000)

print("数据生成成功! 🎉")
print(f"数据形状: X={X.shape}")
print(f"处理组比例: {T.mean():.2%}")
print(f"\n真实因果效应:")
print(f"  真实 ATE: {np.mean(Y1 - Y0):.4f}")
print(f"  真实 ATT: {np.mean((Y1 - Y0)[T == 1]):.4f}")
print(f"\n朴素估计（有偏！）:")
naive_ate = Y[T == 1].mean() - Y[T == 0].mean()
print(f"  朴素 ATE: {naive_ate:.4f}")
print(f"  偏差: {naive_ate - np.mean(Y1 - Y0):.4f}")

---

# Part 4: 对比实验 - TARNet vs DragonNet

现在让我们训练并对比两个模型的性能！

In [ ]:
# 练习 4.1: 统一训练函数

def train_model(
    model: nn.Module,
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    model_type: str = 'tarnet',  # 'tarnet' or 'dragonnet'
    n_epochs: int = 200,
    batch_size: int = 64,
    learning_rate: float = 1e-3,
    alpha: float = 1.0,
    beta: float = 1.0,
    verbose: bool = True
) -> Tuple[nn.Module, dict]:
    """
    统一的模型训练函数
    
    Parameters:
    -----------
    model_type: 'tarnet' (只用 factual loss) 或 'dragonnet' (完整损失)
    alpha: 倾向得分损失权重 (仅 dragonnet)
    beta: targeted regularization 权重 (仅 dragonnet)
    """
    # 数据准备
    X_tensor = torch.FloatTensor(X)
    T_tensor = torch.FloatTensor(T)
    Y_tensor = torch.FloatTensor(Y)
    
    dataset = TensorDataset(X_tensor, T_tensor, Y_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # 优化器
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # 训练历史
    history = {
        'total_loss': [],
        'factual_loss': [],
        'propensity_loss': [],
        'targeted_loss': [],
        'epsilon': []
    }
    
    # 训练循环
    for epoch in range(n_epochs):
        epoch_losses = {k: 0 for k in ['total_loss', 'factual_loss', 'propensity_loss', 'targeted_loss']}
        n_batches = 0
        
        for batch_x, batch_t, batch_y in dataloader:
            optimizer.zero_grad()
            
            # 前向传播
            if model_type == 'dragonnet':
                y0_pred, y1_pred, propensity, epsilon, _ = model(batch_x)
                
                # DragonNet 完整损失
                losses = dragonnet_loss(
                    batch_y, batch_t, y0_pred, y1_pred,
                    propensity, epsilon, alpha=alpha, beta=beta
                )
                loss = losses['total']
                
                # 记录各项损失
                epoch_losses['total_loss'] += loss.item()
                epoch_losses['factual_loss'] += losses['factual'].item()
                epoch_losses['propensity_loss'] += losses['propensity'].item()
                epoch_losses['targeted_loss'] += losses['targeted'].item()
            else:
                # TARNet 只用 factual loss
                y0_pred, y1_pred, _ = model(batch_x)
                loss = compute_factual_loss(batch_y, batch_t, y0_pred, y1_pred)
                
                epoch_losses['total_loss'] += loss.item()
                epoch_losses['factual_loss'] += loss.item()
            
            # 反向传播
            loss.backward()
            optimizer.step()
            
            n_batches += 1
        
        # 记录平均损失
        for k in epoch_losses.keys():
            history[k].append(epoch_losses[k] / n_batches)
        
        # 记录 epsilon (仅 dragonnet)
        if model_type == 'dragonnet':
            history['epsilon'].append(model.epsilon.item())
        
        if verbose and (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{n_epochs}, Loss: {history['total_loss'][-1]:.4f}")
    
    return model, history

In [ ]:
# 练习 4.2: 训练 TARNet

print("="*60)
print("训练 TARNet (只用 Factual Loss)")
print("="*60)

tarnet_model = SimpleTARNet(input_dim=X.shape[1])
tarnet_model, tarnet_history = train_model(
    tarnet_model, X, T, Y,
    model_type='tarnet',
    n_epochs=200,
    batch_size=64,
    verbose=True
)

print(f"\nTARNet 训练完成! 最终损失: {tarnet_history['total_loss'][-1]:.4f}")

In [ ]:
# 练习 4.3: 训练 DragonNet

print("="*60)
print("训练 DragonNet (完整损失)")
print("="*60)

dragonnet_model = SimpleDragonNet(input_dim=X.shape[1])
dragonnet_model, dragonnet_history = train_model(
    dragonnet_model, X, T, Y,
    model_type='dragonnet',
    n_epochs=200,
    batch_size=64,
    alpha=1.0,
    beta=1.0,
    verbose=True
)

print(f"\nDragonNet 训练完成! 最终损失: {dragonnet_history['total_loss'][-1]:.4f}")
print(f"最终 Epsilon: {dragonnet_history['epsilon'][-1]:.4f}")

In [ ]:
# 练习 4.4: 评估函数

def evaluate_model(
    model: nn.Module,
    X: np.ndarray,
    Y0: np.ndarray,
    Y1: np.ndarray,
    model_type: str = 'tarnet'
) -> dict:
    """
    评估模型性能
    """
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X)
        
        if model_type == 'dragonnet':
            y0_pred, y1_pred, _, _, _ = model(X_tensor)
        else:
            y0_pred, y1_pred, _ = model(X_tensor)
        
        y0_pred = y0_pred.numpy().squeeze()
        y1_pred = y1_pred.numpy().squeeze()
    
    # 真实 ITE
    ite_true = Y1 - Y0
    ite_pred = y1_pred - y0_pred
    
    # PEHE (Precision in Estimation of Heterogeneous Effect)
    pehe = np.sqrt(np.mean((ite_true - ite_pred) ** 2))
    
    # ATE 误差
    ate_true = np.mean(ite_true)
    ate_pred = np.mean(ite_pred)
    ate_error = np.abs(ate_true - ate_pred)
    
    # ITE 相关性
    ite_corr = np.corrcoef(ite_true.flatten(), ite_pred.flatten())[0, 1]
    
    return {
        'pehe': pehe,
        'ate_true': ate_true,
        'ate_pred': ate_pred,
        'ate_error': ate_error,
        'ite_corr': ite_corr,
        'ite_true': ite_true,
        'ite_pred': ite_pred,
        'y0_pred': y0_pred,
        'y1_pred': y1_pred
    }

# 评估两个模型
tarnet_metrics = evaluate_model(tarnet_model, X, Y0, Y1, model_type='tarnet')
dragonnet_metrics = evaluate_model(dragonnet_model, X, Y0, Y1, model_type='dragonnet')

print("\n" + "="*60)
print("TARNet vs DragonNet 性能对比")
print("="*60)
print(f"\n{'模型':<15} {'PEHE':<15} {'ATE 误差':<15} {'ITE 相关性':<15}")
print("-"*60)
print(f"{'TARNet':<15} {tarnet_metrics['pehe']:<15.4f} {tarnet_metrics['ate_error']:<15.4f} {tarnet_metrics['ite_corr']:<15.4f}")
print(f"{'DragonNet':<15} {dragonnet_metrics['pehe']:<15.4f} {dragonnet_metrics['ate_error']:<15.4f} {dragonnet_metrics['ite_corr']:<15.4f}")

# 计算改进
pehe_improve = (tarnet_metrics['pehe'] - dragonnet_metrics['pehe']) / tarnet_metrics['pehe'] * 100
ate_improve = (tarnet_metrics['ate_error'] - dragonnet_metrics['ate_error']) / tarnet_metrics['ate_error'] * 100

print(f"\n改进:")
print(f"  PEHE: {pehe_improve:+.1f}%")
print(f"  ATE 误差: {ate_improve:+.1f}%")

In [ ]:
# 可视化对比结果

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 第一行: TARNet 结果
# ITE 散点图
axes[0, 0].scatter(tarnet_metrics['ite_true'], tarnet_metrics['ite_pred'], alpha=0.3, s=10, c='#3498db')
min_val = min(tarnet_metrics['ite_true'].min(), tarnet_metrics['ite_pred'].min())
max_val = max(tarnet_metrics['ite_true'].max(), tarnet_metrics['ite_pred'].max())
axes[0, 0].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
axes[0, 0].set_xlabel('True ITE', fontsize=12)
axes[0, 0].set_ylabel('Predicted ITE', fontsize=12)
axes[0, 0].set_title(f'TARNet: ITE Prediction (r={tarnet_metrics["ite_corr"]:.3f})', fontsize=13, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# ITE 误差分布
ite_error_tarnet = tarnet_metrics['ite_pred'] - tarnet_metrics['ite_true']
n_bins = min(50, max(20, len(ite_error_tarnet) // 40))  # 动态调整 bins
axes[0, 1].hist(ite_error_tarnet, bins=n_bins, alpha=0.7, color='#3498db', edgecolor='black')
axes[0, 1].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero Error')
axes[0, 1].axvline(x=ite_error_tarnet.mean(), color='orange', linestyle='-', linewidth=2, 
                   label=f'Mean: {ite_error_tarnet.mean():.3f}')
axes[0, 1].set_xlabel('ITE Prediction Error', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title(f'TARNet: ITE Error Distribution (PEHE={tarnet_metrics["pehe"]:.3f})', 
                     fontsize=13, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 训练曲线
axes[0, 2].plot(tarnet_history['total_loss'], 'b-', linewidth=2, label='Total Loss')
axes[0, 2].set_xlabel('Epoch', fontsize=12)
axes[0, 2].set_ylabel('Loss', fontsize=12)
axes[0, 2].set_title('TARNet: Training Loss', fontsize=13, fontweight='bold')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# 第二行: DragonNet 结果
# ITE 散点图
axes[1, 0].scatter(dragonnet_metrics['ite_true'], dragonnet_metrics['ite_pred'], alpha=0.3, s=10, c='#e74c3c')
min_val = min(dragonnet_metrics['ite_true'].min(), dragonnet_metrics['ite_pred'].min())
max_val = max(dragonnet_metrics['ite_true'].max(), dragonnet_metrics['ite_pred'].max())
axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
axes[1, 0].set_xlabel('True ITE', fontsize=12)
axes[1, 0].set_ylabel('Predicted ITE', fontsize=12)
axes[1, 0].set_title(f'DragonNet: ITE Prediction (r={dragonnet_metrics["ite_corr"]:.3f})', 
                     fontsize=13, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# ITE 误差分布
ite_error_dragon = dragonnet_metrics['ite_pred'] - dragonnet_metrics['ite_true']
n_bins = min(50, max(20, len(ite_error_dragon) // 40))  # 动态调整 bins
axes[1, 1].hist(ite_error_dragon, bins=n_bins, alpha=0.7, color='#e74c3c', edgecolor='black')
axes[1, 1].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero Error')
axes[1, 1].axvline(x=ite_error_dragon.mean(), color='orange', linestyle='-', linewidth=2, 
                   label=f'Mean: {ite_error_dragon.mean():.3f}')
axes[1, 1].set_xlabel('ITE Prediction Error', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title(f'DragonNet: ITE Error Distribution (PEHE={dragonnet_metrics["pehe"]:.3f})', 
                     fontsize=13, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# 损失组成
axes[1, 2].plot(dragonnet_history['factual_loss'], 'g-', label='Factual', linewidth=2)
axes[1, 2].plot(dragonnet_history['propensity_loss'], 'r-', label='Propensity', linewidth=2)
if len(dragonnet_history['targeted_loss']) > 0 and any(dragonnet_history['targeted_loss']):
    axes[1, 2].plot(dragonnet_history['targeted_loss'], 'purple', label='Targeted', linewidth=2)
axes[1, 2].set_xlabel('Epoch', fontsize=12)
axes[1, 2].set_ylabel('Loss', fontsize=12)
axes[1, 2].set_title('DragonNet: Loss Components', fontsize=13, fontweight='bold')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 对比柱状图

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PEHE 对比
models = ['TARNet', 'DragonNet']
pehe_values = [tarnet_metrics['pehe'], dragonnet_metrics['pehe']]
colors = ['#3498db', '#e74c3c']

bars1 = axes[0].bar(models, pehe_values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
axes[0].set_ylabel('PEHE', fontsize=12)
axes[0].set_title('PEHE 对比 (越低越好)', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# 添加数值标签
for bar, val in zip(bars1, pehe_values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                f'{val:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=11)

# ATE Error 对比
ate_errors = [tarnet_metrics['ate_error'], dragonnet_metrics['ate_error']]

bars2 = axes[1].bar(models, ate_errors, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
axes[1].set_ylabel('ATE Error', fontsize=12)
axes[1].set_title('ATE 误差对比 (越低越好)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# 添加数值标签
for bar, val in zip(bars2, ate_errors):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                f'{val:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

---

# Part 5: 何时选择哪个模型？

## 决策树

```
开始
  |
  v
是否存在强混淆？
  |
  +-- 否 (RCT或弱混淆) --> TARNet
  |                        - 更简单
  |                        - 更快训练
  |                        - 足够准确
  |
  +-- 是 (强混淆) --> 处理分配是否高度不平衡？
                        |
                        +-- 否 --> TARNet 或 DragonNet 都可
                        |          (DragonNet 略好)
                        |
                        +-- 是 --> DragonNet
                                   - 倾向得分头有帮助
                                   - Targeted Reg 减少偏差
```

## 选择指南表

| 场景 | 推荐模型 | 原因 |
|------|----------|------|
| **随机对照试验 (RCT)** | TARNet | 无混淆，无需倾向得分 |
| **观察性数据 + 弱混淆** | TARNet | 简单有效，训练快 |
| **观察性数据 + 强混淆** | DragonNet | 倾向得分正则化有帮助 |
| **处理分配高度不平衡** | DragonNet | Targeted Reg 更稳健 |
| **数据量很小 (< 500)** | TARNet | 避免过拟合 |
| **高维特征 (图像/文本)** | DragonNet | 更好的表示学习 |

## 超参数调优建议

### TARNet
- `hidden_dim`: 50-200
- `repr_dim`: 25-100
- `learning_rate`: 1e-4 ~ 1e-2
- `batch_size`: 32-128

### DragonNet
- 基础参数同 TARNet
- `alpha` (倾向得分权重): 0.5-2.0
  - 混淆越强，alpha 越大
  - 建议从 1.0 开始
- `beta` (targeted reg 权重): 0.5-2.0
  - 处理分配越不平衡，beta 越大
  - 建议从 1.0 开始

---

# 思考题

请在下面的单元格中写下你的答案：

# 思考题 1: 为什么 TARNet 需要共享表示层？为什么不给两个头分别的特征提取器？

**TODO**: 请思考以下问题并写出你的答案
- 提示：考虑样本效率、知识迁移、正则化等角度
- 对比 TARNet 和 T-Learner 的区别

<details>
<summary>点击查看参考答案</summary>

共享表示层的三大优势:

1. **样本效率提升**:
   - T=1的样本只能训练Y(1)头，T=0的样本只能训练Y(0)头
   - 但共享层Φ(X)可以从所有样本中学习
   - 相当于变相增加了训练数据

2. **知识迁移**:
   - Y(0)和Y(1)往往有共同的预测因子（比如用户基础特征）
   - 共享表示允许两个头之间知识迁移
   - 类似Multi-Task Learning的硬参数共享

3. **正则化作用**:
   - 强迫模型学习对两组都有用的通用特征
   - 防止过拟合到某一组的噪声

如果分别使用特征提取器（T-Learner）:
- 数据利用率低（每个模型只用一半数据）
- 容易过拟合
- 但异质性建模更灵活（两个完全独立的模型）

决策: 数据量小/特征多 → TARNet; 数据量大/异质性极强 → T-Learner

</details>

# 思考题 2: Factual Loss 和普通的监督学习损失有什么区别？

**TODO**: 请思考并回答
- 提示：考虑哪些样本参与损失计算？为什么不能用普通回归？

<details>
<summary>点击查看参考答案</summary>

核心区别在于**哪些样本参与损失计算**:

**普通监督学习**:
- 所有样本都有标签Y
- 损失 = MSE(Y_pred, Y_true)，对所有样本

**Factual Loss**:
- 每个样本只有一个观测结果（factual）
- T=1的样本: 只在Y(1)头计算损失
- T=0的样本: 只在Y(0)头计算损失
- 损失是"条件性的"

数学形式对比:

普通监督:
$$L = \frac{1}{N} \sum_i (Y_i - f(X_i, T_i))^2$$

Factual Loss:
$$L = \frac{1}{N} \sum_i (Y_i - [T_i \cdot \mu_1(X_i) + (1-T_i) \cdot \mu_0(X_i)])^2$$

关键洞察:
- Factual Loss实际上是**半监督学习**
- Y(0)对于T=1的样本是"缺失"的
- Y(1)对于T=0的样本是"缺失"的
- 共享表示Φ(X)起到了知识迁移的作用

为什么不能用普通监督学习?
- 因为我们想估计Y(0)和Y(1)的差异（ITE）
- 普通回归只给出Y = f(X, T)，无法分离反事实

</details>

# 思考题 3: DragonNet 的倾向得分头在模型中起什么作用？为什么不单独训练一个倾向得分模型？

**TODO**: 请思考并回答
- 提示：考虑联合训练的优势、多任务学习、梯度反向传播

<details>
<summary>点击查看参考答案</summary>

倾向得分头的三大作用:

1. **正则化共享表示**:
   - 强迫Φ(X)同时学习"预测Y"和"预测T"
   - 这意味着表示层必须捕获影响处理分配的因素
   - 这些因素往往也是混淆因子!

2. **信息共享与协同训练**:
   - 联合训练让三个任务相互增强
   - 倾向得分头的梯度也会更新共享表示
   - 类似Multi-Task Learning的效果

3. **Targeted Regularization的基础**:
   - DragonNet的第三项损失需要倾向得分e(X)
   - 如果单独训练，梯度无法反向传播
   - 联合训练实现端到端优化

为什么不单独训练倾向得分模型?

**分离训练的问题**:
- 倾向得分模型和结果模型各自优化，可能不一致
- 无法端到端优化Targeted Regularization
- 失去了多任务学习的正则化benefit

**联合训练的优势**:
- 共享表示Φ(X)被多个目标约束
- 倾向得分头起到"指导"作用，告诉表示层什么是重要特征
- 理论上更接近双重鲁棒性估计器

类比: 像是在学习时同时做"习题"(预测Y)和"知识点梳理"(理解T的分配机制)，效果比单独做习题更好

</details>

# 思考题 4: Targeted Regularization 中的 h = T/e(X) - (1-T)/(1-e(X)) 的直觉是什么？

**TODO**: 请思考并回答
- 提示：考虑逆倾向加权(IPW)、双重鲁棒性、民意调查的类比

<details>
<summary>点击查看参考答案</summary>

这个公式来自**逆倾向加权(IPW)**和 **双重鲁棒性**理论。

直觉解释 - 民意调查的类比:

想象你在做民意调查，但不同人群被调查到的概率不同:

$$h = \frac{T}{e(X)} - \frac{1-T}{1-e(X)}$$

**第一项 T/e(X)** (被处理组):
- e(X)小的人 → 权重大
- 意思: "本来不太可能被处理，但实际被处理了"的样本更重要
- 类比: 不爱接电话的年轻人如果接了电话，他们的意见要加权

**第二项 -(1-T)/(1-e(X))** (控制组):
- e(X)大的人 → 权重大（取负号）
- 意思: "本来很可能被处理，但实际没被处理"的样本更重要
- 类比: 爱接电话的老年人如果没接电话，这个缺失更需要补偿

**h的作用**:
1. 平衡处理组和控制组的分布差异
2. 给"反常"样本更大权重（高e(X)但T=0，或低e(X)但T=1）
3. 本质是IPW的调整因子

**为什么在Targeted Reg中使用**:

$$L_{TR} = E[(Y - \hat{Y} - \epsilon \cdot h)^2]$$

- 如果模型预测$\hat{Y}$已经很准，ε会学到0
- 如果有系统性偏差，ε·h提供"倾向得分方向"的修正
- 这实现了双重鲁棒性: 结果模型准 or 倾向得分模型准，任一个对，估计就无偏

数学来源:
这来自半参数效率理论中的Efficient Influence Function (EIF)。

</details>

# 思考题 5: epsilon 参数的作用是什么？为什么它是可学习的？

**TODO**: 请思考并回答
- 提示：考虑自适应修正、端到端优化、模型质量指标

<details>
<summary>点击查看参考答案</summary>

epsilon (ε) 是Targeted Regularization中的**可学习标量参数**。

作用:

1. **自适应修正幅度**:
   - ε控制倾向得分调整项h的强度
   - $L_{TR} = E[(Y - \hat{Y} - \epsilon \cdot h)^2]$
   - ε大 → 更依赖倾向得分修正
   - ε小 → 更依赖结果模型预测

2. **双重鲁棒性的桥梁**:
   - 如果结果模型$\hat{Y}$已经很准 → ε会学到≈0
   - 如果结果模型有偏差 → ε学到非零值，用h来补偿
   - 自动平衡两种策略

3. **模型选择指标**:
   - 训练后ε的值反映模型质量
   - ε≈0: 结果模型已经足够准
   - ε较大: 需要倾向得分修正

为什么是可学习的?

**固定ε的问题**:
- 不同数据集最优的ε不同
- 混淆程度强 → 需要大ε
- 混淆程度弱 → 需要小ε

**可学习ε的优势**:
- 端到端优化，自动找到最优修正强度
- 适应不同的数据分布
- 避免手动调参

实现细节:
```python
class DragonNet(nn.Module):
    def __init__(self):
        self.epsilon = nn.Parameter(torch.tensor(0.0))  # 初始化为0
```

训练过程:
- 初始阶段: ε≈0，主要靠结果模型
- 中期: ε开始学习，引入倾向得分修正
- 后期: ε收敛到某个值，反映最优平衡点

监控ε的变化可以理解模型的学习过程!

</details>

# 思考题 6: 在你的实验中，DragonNet 比 TARNet 效果好吗？为什么？

**TODO**: 运行实验后，分析并回答
- 比较PEHE、ATE误差等指标
- 思考数据特性（混淆程度、样本量）对结果的影响

<details>
<summary>点击查看参考答案</summary>

从实验结果看（运行你自己的实验后填写）:

PEHE对比: TARNet = ____, DragonNet = ____
ATE误差对比: TARNet = ____, DragonNet = ____

**如果DragonNet更好**:

原因分析:
1. 数据有强混淆 - 我们生成数据时特意设计了混淆
2. 倾向得分头捕获了处理分配机制
3. Targeted Regularization提供了额外的正则化

**如果差不多**:

可能原因:
1. 数据量不够大，DragonNet的优势没体现出来
2. 混淆不够强，TARNet已经足够
3. 超参数α和β没调好

**如果TARNet更好** (较少见):

可能原因:
1. DragonNet过拟合（参数更多）
2. 倾向得分头学习失败（检查propensity_loss是否收敛）
3. alpha和beta设置不合理

**实验设计的洞察**:

我们的数据生成过程:
```python
# 强混淆: X影响T的分配
propensity = sigmoid(1.0*X[:,0] + 0.8*X[:,1] + 0.6*X[:,2])

# X也影响Y
Y0 = 2.0 + 1.0*X[:,0] + 0.5*X[:,1] + noise
```

这是典型的"X既是混淆因子又是预测因子"场景 → DragonNet应该更好

**进一步实验**:
可以尝试调整:
- 增大alpha（如2.0）→ 更强的倾向得分约束
- 增大beta（如2.0）→ 更强的targeted reg
- 观察epsilon的变化轨迹

</details>

# 思考题 7: 如果处理分配是完全随机的（RCT），DragonNet 还有优势吗？

**TODO**: 请思考并回答
- 提示：考虑RCT中的倾向得分特性、混淆因子的作用

<details>
<summary>点击查看参考答案</summary>

在RCT（随机对照试验）中，**DragonNet的优势会大幅减弱，但不会完全消失**。

**理论分析**:

RCT意味着: $T \perp X$ (处理分配与特征独立)

此时:
1. **无混淆** → 不需要倾向得分调整
2. **e(X) = 常数** (比如0.5) → 倾向得分头学到的是常数
3. **Targeted Reg中的h ≈ 0** → 第三项损失几乎不起作用

DragonNet退化为:
$$L = L_{\text{factual}} + \alpha \cdot L_{\text{propensity}}$$

而$L_{\text{propensity}}$在RCT中只是在拟合一个常数函数。

**可能的小优势**:

1. **正则化效果**:
   - 即使在RCT，多任务学习的正则化仍然有用
   - 倾向得分头可能帮助表示层学习更鲁棒的特征

2. **平衡检验**:
   - 如果倾向得分头学到e(X) ≠ 常数，说明可能有问题
   - 可以用来检验随机化是否成功

**实验验证**:

生成RCT数据:
```python
T = np.random.binomial(1, 0.5, n)  # 完全随机
```

预期结果:
- TARNet和DragonNet性能接近
- DragonNet的epsilon ≈ 0
- propensity_head学到≈0.5的常数

**实践建议**:

RCT场景:
- 优先用TARNet（更简单）
- 如果用DragonNet，设置较小的alpha和beta
- 关注epsilon的值（应该接近0）

观察性研究:
- DragonNet更有优势
- 倾向得分头捕获选择偏差
- Targeted Reg减少偏差

结论: DragonNet是为**混淆/选择偏差**设计的，RCT中其优势不明显。

</details>

---

# 面试题模拟与深度剖析

## 高频面试题集锦

### 题目 1: TARNet 和普通神经网络回归有什么区别？

**标准答案**:

TARNet 与普通神经网络回归的核心区别在于:

1. **架构设计**:
   - 普通回归: X → NN → Y
   - TARNet: X → 共享表示Φ(X) → 双头[μ₀, μ₁]

2. **训练策略**:
   - 普通回归: 直接用所有样本训练Y = f(X, T)
   - TARNet: Factual Loss - 只在观测到的结果上计算损失

3. **反事实推断**:
   - 普通回归: 无法直接给出反事实
   - TARNet: 同时预测Y(0)和Y(1)

4. **样本效率**:
   - 普通回归: 两组样本独立训练
   - TARNet: 共享表示，提高样本效率

**进阶回答** (加分项):

TARNet的Factual Loss实际上是在做**半监督学习**:
- 对于T=1的样本，我们只观测到Y(1)，Y(0)是"缺失"的
- 共享表示Φ(X)允许两组之间的知识迁移
- 这类似于Multi-Task Learning中的硬参数共享

---

### 题目 2: DragonNet 为什么要联合预测倾向得分？

**标准答案**:

DragonNet添加倾向得分头的三大原因:

1. **正则化作用**:
   - 强迫表示层Φ(X)同时学习"预测Y"和"预测T"
   - 防止表示层只优化结果预测而忽略混淆因子

2. **捕获混淆**:
   - 倾向得分e(X) = P(T=1|X)反映了X中哪些因素影响处理分配
   - 这些因素往往也是混淆因子

3. **Targeted Regularization**:
   - 利用倾向得分构造双重鲁棒性的调整项
   - 理论上可以减少偏差(来自TMLE理论)

**数学推导**:

根据双重鲁棒性理论:

$$\tau = \mathbb{E}\left[\frac{T \cdot Y}{e(X)} - \frac{(1-T) \cdot Y}{1-e(X)}\right]$$

DragonNet的Targeted Regularization正是利用这个原理:

$$h_i = \frac{T_i}{\hat{e}(X_i)} - \frac{1-T_i}{1-\hat{e}(X_i)}$$

当结果模型μ₀, μ₁准确或倾向得分模型e(X)准确时，估计都是无偏的。

**面试加分点**:
- 提到TMLE (Targeted Maximum Likelihood Estimation)
- 提到双重鲁棒性
- 能画出DragonNet的三头架构图

---

### 题目 3: CEVAE 的核心假设是什么？

**标准答案**:

CEVAE依赖于**代理变量假设** (Proxy Variable Assumption):

1. **假设内容**:
   - 存在隐变量Z影响X、T、Y
   - 观测到的X包含关于Z的"足够信息"
   - 形式化: $p(Y(t) | X, Z) = p(Y(t) | X)$

2. **识别条件**:
   ```
   Z → X (X是Z的代理)
   Z → T (Z影响处理分配)
   Z → Y (Z影响结果)
   ```

3. **与其他方法的对比**:
   - IPW/PSM假设: 所有混淆都被观测 (Unconfoundedness)
   - IV假设: Z ⊥ Y | T (Z不直接影响Y)
   - CEVAE假设: Z部分观测(通过X)

**为什么这个假设重要**:

如果X完全不包含Z的信息，CEVAE会失败，因为:
- 后验推断q(Z|X,T,Y)无法从X中"还原"Z
- 这时CEVAE退化为盲目猜测

**面试官追问**: "如何验证这个假设？"

答: 
1. 领域知识: 确保X包含相关的代理指标
2. 敏感性分析: 测试不同的隐变量维度
3. 预测检验: 检查X的重构质量

---

### 题目 4: 深度学习因果模型的主要挑战是什么？

**标准答案** (多维度回答):

#### 1. 理论挑战

- **识别性**: 深度模型的非凸优化使得识别条件难以验证
- **过拟合**: 参数多，容易记住噪声而非因果关系
- **可解释性**: 黑盒模型，难以理解学到了什么

#### 2. 实践挑战

- **超参数敏感**: 网络深度、宽度、学习率都会影响结果
- **训练不稳定**: GAN-based方法(如GANITE)容易模式崩溃
- **计算成本**: 比传统方法慢很多

#### 3. 数据挑战

- **样本需求**: 深度模型通常需要更多数据
- **标签噪声**: 对Y的测量误差更敏感
- **分布偏移**: 测试集和训练集分布不同时表现差

#### 4. 方法论挑战

- **评估困难**: 真实ITE无法观测，只能用PEHE等模拟指标
- **基准缺失**: 没有统一的benchmark
- **调参黑箱**: 不同任务需要重新调参

**最佳实践建议**:

1. 从简单模型开始(如TARNet)
2. 使用交叉验证选择超参数
3. 进行消融实验验证每个组件的作用
4. 敏感性分析测试鲁棒性

---

## 进阶面试题

### 题目 5: 如果你要设计一个新的深度因果模型，你会考虑什么？

**答题思路** (展示思维过程):

#### Step 1: 明确问题设定

- 二值处理 vs 连续处理 vs 多值处理？
- 观测数据 vs RCT？
- 是否存在未观测混淆？

#### Step 2: 选择基础架构

- 共享表示 + 多头 (类似TARNet)
- VAE框架 (类似CEVAE)
- GAN框架 (类似GANITE)

#### Step 3: 设计损失函数

核心权衡:
```
L_total = L_prediction + α·L_balance + β·L_regularization
```

- L_prediction: Factual loss
- L_balance: IPM (MMD/Wasserstein)
- L_regularization: 领域知识约束

#### Step 4: 创新点

可能的方向:
1. **注意力机制**: 自动学习哪些特征是混淆因子
2. **元学习**: 快速适应新的因果任务
3. **不确定性量化**: 贝叶斯深度学习
4. **因果图发现**: 联合学习结构和参数

**面试加分点**:
- 引用最新论文 (ICML/NeurIPS)
- 提到具体应用场景
- 讨论trade-offs

---

### 题目 6: TARNet和T-Learner有什么区别？什么时候用哪个？

**对比分析**:

| 维度 | T-Learner | TARNet |
|------|-----------|--------|
| 参数共享 | 无 (两个独立模型) | 有 (共享表示层) |
| 样本效率 | 低 (各用一半数据) | 高 (共享知识) |
| 过拟合风险 | 高 | 中等 |
| 实现复杂度 | 简单 | 中等 |
| 异质性建模 | 灵活 | 受共享表示约束 |

**决策树**:

```
数据量大 (>10k) ?
├─ 是 → 异质性很强？
│       ├─ 是 → T-Learner
│       └─ 否 → TARNet
└─ 否 → 特征维度高？
        ├─ 是 → TARNet (需要正则化)
        └─ 否 → T-Learner 或 TARNet都可
```

---

## 核心数学推导

### 推导 1: TARNet 的 Factual Loss

**目标**: 最小化预测误差

$$\mathcal{L} = \mathbb{E}[(Y - \hat{Y})^2]$$

**关键点**: 我们只观测到factual outcome

$$\hat{Y}^{factual} = \begin{cases} 
\mu_1(\Phi(X)) & \text{if } T = 1 \\
\mu_0(\Phi(X)) & \text{if } T = 0
\end{cases}$$

**展开**:

$$\begin{align}
\mathcal{L} &= \mathbb{E}_{T=1}[(Y - \mu_1(\Phi(X)))^2] \cdot P(T=1) \\
&\quad + \mathbb{E}_{T=0}[(Y - \mu_0(\Phi(X)))^2] \cdot P(T=0)
\end{align}$$

**实现**:

```python
loss = torch.where(T == 1, 
                   (Y - mu1)**2, 
                   (Y - mu0)**2).mean()
```

---

### 推导 2: DragonNet 的 Targeted Regularization

**来源**: TMLE理论

**核心思想**: 利用倾向得分构造一阶无偏估计

**推导**:

1. 真实ATE的efficient influence function (EIF):

$$\psi(X, T, Y) = \frac{T \cdot (Y - \mu_1(X))}{e(X)} - \frac{(1-T) \cdot (Y - \mu_0(X))}{1 - e(X)} + \mu_1(X) - \mu_0(X)$$

2. 一阶条件 (first-order condition):

$$\mathbb{E}[\psi(X, T, Y)] = \tau_{ATE}$$

3. Targeted Regularization惩罚EIF的方差:

$$\mathcal{L}_{TR} = \mathbb{E}\left[\left(Y - \hat{\mu} - \epsilon \cdot h(X, T)\right)^2\right]$$

其中:
$$h(X, T) = \frac{T}{e(X)} - \frac{1-T}{1-e(X)}$$

**直觉**: 
- h(X,T)是倾向得分的"修正项"
- ε是可学习的修正幅度
- 这个损失鼓励模型在倾向得分边界附近更准确

---

### 推导 3: CEVAE 的 ELBO

**目标**: 最大化边际对数似然

$$\log p_\theta(X, T, Y) = \log \int p_\theta(X, T, Y, Z) dZ$$

**问题**: 积分intractable

**解决**: 变分推断，引入q_φ(Z|X,T,Y)

**Jensen不等式**:

$$\begin{align}
\log p_\theta(X, T, Y) &= \log \mathbb{E}_{q_\phi} \left[\frac{p_\theta(X, T, Y, Z)}{q_\phi(Z|X,T,Y)}\right] \\
&\geq \mathbb{E}_{q_\phi} \left[\log \frac{p_\theta(X, T, Y, Z)}{q_\phi(Z|X,T,Y)}\right] \\
&= \mathbb{E}_{q_\phi}[\log p_\theta(X|Z)] + \mathbb{E}_{q_\phi}[\log p_\theta(T|X,Z)] \\
&\quad + \mathbb{E}_{q_\phi}[\log p_\theta(Y|T,X,Z)] - KL(q_\phi || p(Z))
\end{align}$$

**ELBO**:

$$\mathcal{L}_{ELBO} = \mathbb{E}_{q_\phi}[\log p(X,T,Y|Z)] - KL(q_\phi(Z|X,T,Y) || p(Z))$$

**各项解释**:

1. $\log p(X|Z)$: X的重构损失
2. $\log p(T|X,Z)$: T的分类损失
3. $\log p(Y|T,X,Z)$: Y的预测损失
4. KL散度: 正则化项，防止后验崩溃

---

### 推导 4: GANITE 的对抗损失

**Block 1: 反事实生成**

**生成器目标**:

$$\min_{G_{cf}} \mathbb{E}_{X,T,Y,Z}\left[D_{cf}(X, T, Y, G_{cf}(X, T, Y, Z))\right]$$

**判别器目标**:

$$\max_{D_{cf}} \mathbb{E}[D_{cf}(X, T, Y, Y_{cf}^{true})] + \mathbb{E}[\log(1 - D_{cf}(X, T, Y, Y_{cf}^{fake}))]$$

**监督约束** (关键!):

$$\mathcal{L}_{sup} = \mathbb{E}[(Y - G_{cf}(X, T, Y, Z))^2] \quad \text{when generating factual}$$

**总损失**:

$$\mathcal{L}_{G_{cf}} = -\mathbb{E}[\log D_{cf}] + \lambda \cdot \mathcal{L}_{sup}$$

**Block 2: ITE估计**

使用Block 1填补的完整潜在结果训练ITE生成器:

$$\hat{\tau}(X) = G_{ITE}(X)$$

约束: $\hat{Y}(1) - \hat{Y}(0) = \hat{\tau}(X)$

---

## 实战技巧总结

### 调参经验

| 超参数 | TARNet | DragonNet | CEVAE | GANITE |
|--------|--------|-----------|-------|--------|
| 学习率 | 1e-3 | 1e-3 | 1e-4 | 1e-4 (G), 1e-3 (D) |
| 批大小 | 64-256 | 128-512 | 64-128 | 128-256 |
| 隐藏层维度 | 100-300 | 200-400 | 200-500 | 64-128 |
| 正则化权重α | - | 0.5-2.0 | - | - |
| KL权重β | - | 0.5-2.0 | 0.1-1.0 | - |

### 训练监控指标

1. **收敛性**:
   - Train/Val loss曲线
   - 梯度范数

2. **平衡性**:
   - MMD或Wasserstein距离
   - SMD (每个特征维度)

3. **预测质量**:
   - Factual预测的MSE
   - 倾向得分的AUC (DragonNet)

4. **反事实合理性** (如有真值):
   - PEHE
   - ATE误差

---

*恭喜你完成了深度因果模型的面试题和数学推导！* 🎓